In [11]:
# --------------------------------------------
# Grid erstellen und anzeigen
# --------------------------------------------

# Funktion, um das Grid zu erstellen mit Hindernissen und Ziel
def create_grid(num_obstacles):
    
    global grid, grid_size, robot_pos, object_pos, obstacles # globale Variablen bekannt machen
    obstacles = []  # Liste für Hindernis-Koordinaten

    # Raster-Größe (10x10; Liste mit 10 Row-Listen als Ansatz) 
    grid_size = 10

    # Leeres Grid erstellen als Liste
    grid = []
    
    i = 0 # Zählvariable 1
    while i < grid_size:
        row = [] # jede Zeile als eigene Liste
        j = 0 # Zählvariable 2

        while j < grid_size:
            row.append(".") # Jede Zeile 10 Punkte als "leere Felder"
            j += 1

        grid.append(row) # Jede Zeile als Liste in Main-Liste "grid" einfügen 
        i += 1

    # Roboter-Startposition setzen (immer links oben 0,0)
    robot_pos = [0, 0]
    grid[robot_pos[1]][robot_pos[0]] = "R"  # "R" für Roboter setzen an Position 0,0

    # Objekt zufällig platzieren
    import random
    while True:
        ox = random.randint(0, grid_size - 1)
        oy = random.randint(0, grid_size - 1)
        if [ox, oy] != robot_pos:  # Stelle darf nicht gleich Roboter-Startpunkt sein
            object_pos = [ox, oy]
            grid[oy][ox] = "O"  # "O" als Objekt
            break

    # Hindernisse zufällig setzen, je nach Schwierigkeitsgrad
    k = 0 # Zählvariable
    while k < num_obstacles:
        ox = random.randint(0, grid_size - 1)
        oy = random.randint(0, grid_size - 1)
        
        if [ox, oy] != robot_pos and [ox, oy] != object_pos and grid[oy][ox] == ".":
            grid[oy][ox] = "#"  # "#" als Symbol für Hindernis
            obstacles.append([ox, oy])
            k += 1


# Funktion, um das Grid mit Hindernissen anzuzeigen
def display_grid():
    
    # Beschriftungen 
    print("")
    print("Auto-Modus: Bewegungsraum mit Hindernissen:")
    print("")

    # X-Achse oben anzeigen
    header = "    "
    i = 0 # Zählvariable
    
    while i < grid_size:
        header = header + str(i) + "  "
        i = i + 1
    print(header)

    # Trennlinie
    print("   " + "---" * grid_size)

    # Alle Zeilen mit Y-Wert anzeigen
    y = 0 # Zählvariable 1
    
    while y < grid_size:
        row_text = str(y) + " | "
        
        x = 0 # Zählvariable 2
        
        while x < grid_size:
            row_text = row_text + grid[y][x] + "  "
            x = x + 1
        print(row_text)
        y = y + 1


In [12]:
# --------------------------------------------
# A*-Algorithmus zur Pfadplanung
# --------------------------------------------

# Funktion, um den optimalen Pfad mit A* zu berechnen
def find_path_a_star(start, goal):
    # Start- und Zielknoten als Tupel (x, y)
    start_node = tuple(start)
    goal_node = tuple(goal)

    # Noch zu überprüfende Knoten mit f-Wert (f = g + h)
    open_set = {start_node: 0}

    # Woher kommt jeder Knoten (zurückverfolgen des Pfades)
    came_from = {}

    # g-Werte: tatsächliche Kosten vom Start bis zum aktuellen Knoten
    g_score = {start_node: 0}

    # Bereits geprüfte Knoten
    closed_set = {}

    # Bewegungsrichtung als x, y Paare (rechts, links, runter, hoch)
    directions = [(1,0), (-1,0), (0,1), (0,-1)]

    # Heuristik: Manhattan-Distanz (Gitter-Logik ohne Diagonale)
    def heuristic(a, b):
        return abs(a[0] - b[0]) + abs(a[1] - b[1])

    # Solange offene Knoten vorhanden sind weitersuchen
    while open_set:
        # Knoten mit geringstem f-Wert auswählen
        current = min(open_set, key=open_set.get)

        # Wenn Ziel erreicht Pfad zurückverfolgen
        if current == goal_node:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.reverse()  # Reihenfolge umdrehen: Start → Ziel
            return path

        # Aktuellen Knoten aus der offenen Liste entfernen und in die geschlossene Liste verschieben
        closed_set[current] = open_set[current]
        del open_set[current]

        # Nachbarn anschauen (oben, unten, links, rechts)
        for direction in directions:
            neighbor = (current[0] + direction[0], current[1] + direction[1])

            # Prüfen, ob Nachbar innerhalb des Grids liegt
            if 0 <= neighbor[0] < grid_size and 0 <= neighbor[1] < grid_size:
                # Prüfen, ob Hindernis im Weg ist
                if grid[neighbor[1]][neighbor[0]] == "#":
                    continue  # Hindernis wird übersprungen

                # Vorläufige g-Kosten berechnen
                tentative_g_score = g_score[current] + 1

                # Wenn Knoten bereits besucht wurde, überspringen
                if neighbor in closed_set:
                    continue

                # Wenn Knoten noch nicht bekannt oder besserer Pfad gefunden
                if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_score = tentative_g_score + heuristic(neighbor, goal_node)
                    open_set[neighbor] = f_score

    # Wenn kein Pfad gefunden wurde None zurückgeben
    return None


In [13]:
# --------------------------------------------
# Auto-Modus starten - Aufruf A* und Pfadvisualisierung 
# --------------------------------------------

# Funktion für den Auto-Modus (Pfadplanung + Ausführung)
def start_auto_mode():
    
    print("")  # Leerzeile für bessere Lesbarkeit
    print("🟢 Auto-Modus gestartet: Der Roboter plant nun selbst den Weg zum Zielobjekt.")
    print("Bitte warten...")

    # Pfad mit A*-Algorithmus berechnen
    path = find_path_a_star(robot_pos, object_pos)

    # Prüfen, ob ein Pfad gefunden wurde
    if path is None:
        print("\033[1;31m❌ Kein Pfad zum Ziel gefunden! Bitte starte einen neuen Task mit anderen Hindernissen.\033[0m")
        return

    # Pfad im Grid visualisieren (Pfadvorschau)
    for pos in path:
        x, y = pos
        if grid[y][x] == ".":
            grid[y][x] = "*"  # Pfadvorschau

    display_grid()  # Grid mit geplantem Pfad anzeigen

    # Nachfrage beim User: Pfad ausführen?
    auswahl = input("Soll der berechnete Pfad jetzt ausgeführt werden? (ja/nein): ")

    if auswahl.lower() == "ja":
        print("Bewegung wird ausgeführt - Roboter bewegt sich zum Ziel...")

        # Startposition merken
        start_pos = robot_pos.copy()

        # Startposition mit X markieren
        grid[start_pos[1]][start_pos[0]] = "\033[1;32mX\033[0m"

        # Wegpunkte mit grünem Stern markieren
        for pos in path[:-1]:
            x, y = pos
            grid[y][x] = "\033[1;32m*\033[0m"

        # Roboter auf Zielposition setzen mit grünem "R"
        x, y = path[-1]
        grid[y][x] = "\033[1;32mR\033[0m"

        display_grid()  # Fertigen Pfad und Roboter anzeigen

        print("\033[1;32m✅ Erfolgreiche Durchführung - Der Roboter hat das Zielobjekt erreicht!\033[0m")

    else:
        print("Pfad wurde **nicht** ausgeführt. Du kannst jetzt neue Befehle eingeben oder den Task neu starten.")


In [14]:
# --------------------------------------------
# Main-Programm mit Ablauf und User-Interaktion
# --------------------------------------------

def main():
    
    print("\nWillkommen im Auto-Modus zur autonomem Wegplanung intelligenter Roboter!\n")
    
    while True:
        # Auswahl des Schwierigkeitsgrads durch den Nutzer
        print("Wähle den Schwierigkeitsgrad für den Navigations-Task:")
        print("1 = leicht (wenige Hindernisse)")
        print("2 = mittel (mittlere Anzahl Hindernisse)")
        print("3 = schwer (viele Hindernisse)")
        
        schwierigkeit = input("Deine Wahl (1/2/3): ")
        if schwierigkeit.lower() == "abbruch":
            print("\n\033[1;33mProgramm wird abgebrochen - Auto-Modus wird geschlossen.\033[0m")
            break
        
        # Anzahl der Hindernisse je nach Modus festlegen
        if schwierigkeit == "1":
            anzahl_hindernisse = 10
        elif schwierigkeit == "2":
            anzahl_hindernisse = 20
        elif schwierigkeit == "3":
            anzahl_hindernisse = 30
        else:
            print("Ungültige Eingabe! Bitte nochmal versuchen.")
            continue  # zurück zum Anfang der Schleife

        # Grid mit Hindernissen erstellen
        create_grid(anzahl_hindernisse)
        display_grid()

        # Auto-Modus starten
        start_modus = input("\nMöchtest du den Auto-Modus starten? (ja/nein): ")
        if start_modus.lower() == "abbruch":
            print("\n\033[1;33mProgramm wird abgebrochen - Auto-Modus wird geschlossen.\033[0m")
            break
        if start_modus.lower() == "ja":
            print("\n\033[1;34mAuto-Modus gestartet – der Roboter berechnet den optimalen Pfad ...\033[0m")
            start_auto_mode()
        else:
            print("Auto-Modus wurde nicht gestartet.")

        # Neuer Task? Ja = von vorne, Nein = Programm beenden
        neue_runde = input("\nMöchtest du einen neuen Task starten? (ja/nein): ")
        if neue_runde.lower() == "abbruch":
            print("\n\033[1;33mProgramm wird abgebrochen - Auto-Modus wird geschlossen.\033[0m")
            break
        if neue_runde.lower() != "ja":
            print("\n\033[1;33mProgramm beendet. Bewegungsplanung und -Ausführung ist abgeschlossen.\033[0m")
            break


In [15]:
# --------------------------------------------
# Aufruf des Hauptprogramms (Startpunkt)
# --------------------------------------------

main()



Willkommen im Auto-Modus zur autonomem Wegplanung intelligenter Roboter!

Wähle den Schwierigkeitsgrad für den Navigations-Task:
1 = leicht (wenige Hindernisse)
2 = mittel (mittlere Anzahl Hindernisse)
3 = schwer (viele Hindernisse)


Deine Wahl (1/2/3):  3



Auto-Modus: Bewegungsraum mit Hindernissen:

    0  1  2  3  4  5  6  7  8  9  
   ------------------------------
0 | R  .  .  .  .  .  .  .  .  .  
1 | #  .  #  .  #  .  .  .  #  .  
2 | .  .  .  .  .  .  .  .  #  #  
3 | .  #  #  .  .  .  .  #  .  .  
4 | .  .  O  .  .  .  .  #  #  .  
5 | .  .  #  .  .  .  .  .  #  #  
6 | .  .  #  .  .  .  .  #  .  .  
7 | .  #  .  .  #  #  .  #  .  .  
8 | .  #  .  .  .  .  #  #  #  .  
9 | .  .  #  #  .  .  #  #  #  #  



Möchtest du den Auto-Modus starten? (ja/nein):  ja



Auto-Modus gestartet – der Roboter berechnet den optimalen Pfad ...

🟢 Auto-Modus gestartet: Der Roboter plant nun selbst den Weg zum Zielobjekt.
Bitte warten...

Auto-Modus: Bewegungsraum mit Hindernissen:

    0  1  2  3  4  5  6  7  8  9  
   ------------------------------
0 | R  *  .  .  .  .  .  .  .  .  
1 | #  *  #  .  #  .  .  .  #  .  
2 | .  *  *  *  .  .  .  .  #  #  
3 | .  #  #  *  .  .  .  #  .  .  
4 | .  .  O  *  .  .  .  #  #  .  
5 | .  .  #  .  .  .  .  .  #  #  
6 | .  .  #  .  .  .  .  #  .  .  
7 | .  #  .  .  #  #  .  #  .  .  
8 | .  #  .  .  .  .  #  #  #  .  
9 | .  .  #  #  .  .  #  #  #  #  


...


Auto-Modus gestartet – der Roboter berechnet den optimalen Pfad ...

🟢 Auto-Modus gestartet: Der Roboter plant nun selbst den Weg zum Zielobjekt.
Bitte warten...

Auto-Modus: Bewegungsraum mit Hindernissen:

    0  1  2  3  4  5  6  7  8  9  
   ------------------------------
0 | R  #  .  .  .  .  .  .  .  #  
1 | *  *  *  *  #  .  .  .  #  .  
2 | .  .  .  *  *  *  #  #  .  .  
3 | .  .  .  #  .  *  *  #  .  .  
4 | .  .  .  .  .  #  *  #  #  #  
5 | .  .  .  .  .  .  *  *  .  .  
6 | .  .  #  .  .  .  .  *  .  .  
7 | .  .  .  .  .  .  #  *  #  #  
8 | .  .  .  .  #  .  .  *  .  .  
9 | .  .  .  #  #  .  .  O  .  #  


Soll der berechnete Pfad jetzt ausgeführt werden? (ja/nein):  ja


Bewegung wird ausgeführt - Roboter bewegt sich zum Ziel...

Auto-Modus: Bewegungsraum mit Hindernissen:

    0  1  2  3  4  5  6  7  8  9  
   ------------------------------
0 | X  #  .  .  .  .  .  .  .  #  
1 | *  *  *  *  #  .  .  .  #  .  
2 | .  .  .  *  *  *  #  #  .  .  
3 | .  .  .  #  .  *  *  #  .  .  
4 | .  .  .  .  .  #  *  #  #  #  
5 | .  .  .  .  .  .  *  *  .  .  
6 | .  .  #  .  .  .  .  *  .  .  
7 | .  .  .  .  .  .  #  *  #  #  
8 | .  .  .  .  #  .  .  *  .  .  
9 | .  .  .  #  #  .  .  R  .  #  
✅ Erfolgreiche Durchführung - Der Roboter hat das Zielobjekt erreicht!



Möchtest du einen neuen Task starten? (ja/nein):  nein



Programm beendet. Bewegungsplanung und -Ausführung ist abgeschlossen.
